# Procesamiento Digital de Imágenes - Examen Final

## Instrucciones

1.	El examen consta de 1 pregunta y tendrá 1 semana para resolverla con su equipo del trabajo final (en caso de sobrar uno o dos alumnos pueden crearse, máximo dos equipos de 2 integrantes).
2.	El trabajo será entregado, en el aula virtual, hasta las 7:59:59 am del viernes 8 de diciembre. En seguida habrá una exposición, máximo 10 minutos por grupo, de los grupos de trabajo de 8 a 10 am. **BAJO NINGUN MOTIVO SE ACEPTARAN EXAMENES FUERA DEL LIMITE DE TIEMPO INDICADO**
3.	El examen cuenta con un docente académico, el cual estará conectado durante los primeros 20 minutos del examen.
4.	Las dudas conceptuales sobre el examen han de presentarse dentro de los primeros 20 minutos mediante un correo al profesor GONZALEZ VALENZUELA, RICARDO EUGENIO a pcsirgon@upc.edu.pe.
5.	Los inconvenientes técnicos pueden presentarse a pasado los primeros 20 minutos, puede comunicarlo al profesor GONZALEZ VALENZUELA, RICARDO EUGENIO a pcsirgon@upc.edu.pe.
6.	El profesor en mención solo recibirá correos provenientes de las cuentas UPC, de ninguna manera se recibirán correos de cuentas públicas. 
7.	Ante problemas técnicos, debe de forma obligatoria adjuntar evidencias del mismo, como capturas de pantalla, videos, fotos, etc. Siendo requisito fundamental que, en cada evidencia se pueda apreciar claramente la fecha y hora del sistema operativo del computador donde el alumno está rindiendo el examen. 
8.	Los correos sobre problemas técnicos se recibirán hasta 15 minutos luego de culminado el examen.


## Integrantes

*   Alumno 1: <font color='green'> u20181a010 - Joaquin Adrian Galvan Diaz</font><br>
*   Alumno 2: <font color='green'> u201818067 - Dante Brandon Moreno Carhuacusma</font><br>

## Caso de Estudio - Video Summarization

1. Descargar y/o crear videos de 10 a 15 minutos y efectuar un resumen de tiempo en los mismos.

2. Aplicando únicamente técnicas de procesamiento digital de imágenes, segmentar los objetos que aparecen en diferentes instantes y sobreponerlos en una cantidad muy inferior de frames etiquetando cada objeto con los insantes de tiempo en que aparece en el video.

3. En el siguiente [video](https://www.youtube.com/watch?v=gk3qTMlcadk), podrá tener un mejor concepto de lo solicitado. 

4.	Documente sus métodos y elecciones. Explique su metodología. Codifique su solución. Obtenga resultados y realice comparaciones. Redacte sus conclusiones.


## Resolución

### Metodología (6 puntos)

<font color='green'>Aquí **enumere** y **explique** los pasos de su metodología </font>

#### Primer método: Diferencia de imágenes y ROIs

En nuestro primer método, asumimos que el primer frame del video está "limpio", es decir, es la base sobre la cual detectar los nuevos objetos en la imagen. Definimos entonces:

 * $A$: Primer frame del video.
 * $B_{n}$: n-ésimo frame del video.

La primera tarea es identificar regiones de interés (ROIs) donde ocurran cambios en el video. Luego se podrá utilizar esta información para calcular juntar los objetos en el resumen. El algoritmo de es:

1. Por cada $B_{n}$:
> $ROIs$: Arreglo que guardará las regiones de interés en el frame.
> - 1.1. Hallar la diferencia: $D_{n}$ = $A$ - $B_{n}$. **Si** $B_{n}$ es similar a $A$, $D_{n}$ estará compuesto en su mayoría por $0s$.
> - 1.2. Aplicar filtro de la media o mediana sobre $D_{n}$, ya que puede ser afectado por el rudio
> - 1.3. Binarizar o Aplicar un operador clásico de Edge Detection sobre $D_{n}$
> - 1.4. $contornos \leftarrow findContours(D_{n})$
> - 1.5. Por cada $c$ en $contornos$:
>> * Agregar rectángulo envolvente de $c$ al arreglo $ROIs$

> *TODO*: Mantener el mismo ID del objeto para los diferentes frames, quizá con un Match Template entre ROIs (?) o solo asumiendo que mientras los ROIS de $D_{n}$ y $D_{n+1}$ choquen, son el mismo objeto. Cuando el objeto "salga" de la pantalla, esa ROI debería volver a parecerse a A y convertirse en 0s.

### Implementación (6 puntos)

In [1]:
!pip install pafy
!pip install youtube-dl

In [1]:
import cv2
import numpy as np
import pafy
import matplotlib.pyplot as plt

#### Cargar el vídeo

In [2]:
def cv2_imshow(imagen, titulo = "Imagen"):
    cv2.imshow(titulo,imagen)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
url = "https://www.youtube.com/watch?v=CkVJyAKwByw"
video = pafy.new(url)
best = video.getbest(preftype="mp4")

## Reproducir el video

In [4]:
capture = cv2.VideoCapture()
capture.open(best.url)

anterior = None
start = True

while (capture.isOpened()):
    ##Por cada frame del video:
    #capturar y mostrar el frame
    ret, frame = capture.read()
    #cv2.imshow("It's meta",frame)
    frame = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    
    if not start:        
        dif = anterior - frame
        dif = cv2.adaptiveThreshold(frame,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,0)
        cv2.imshow("Diferencia",dif)
    else:
        start = False    
    anterior = frame

    if(cv2.waitKey(20) & 0xFF == ord('q')):
        anterior = dif
        break
capture.release()
cv2.destroyAllWindows()

cv2.imshow("Diferencia",anterior)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Pruebas Dante

In [6]:
anterior.shape

(720, 1280)

In [5]:
def graficarRectDeROIs(imagen, ROIs):
    for roi in ROIs:
        (x,y,w,h) = roi
        cv2.rectangle(imagen, (x,y), (x+w,y+h), (255,0,0), 2)

Capturar los primeros "N" frames para poder testear.
Se baja la resolución para evitar carga

In [6]:
N_Prueba = 240 #Cantidad de frames de prueba

In [7]:
nH = int(anterior.shape[0] // 1.5)
nW = int(anterior.shape[1] // 1.5)
print(nH,nW)

480 853


Captura los frames de prueba

In [8]:
mB = [] # Milisegundos en los que se ubican los frames B_i
B = [] # Frames de Prueba
A = None # Frame Original
capture.open(best.url)
_, A = capture.read() #.......... Frame "Original", se asume que en este no hay "objetos" en pantalla
A = cv2.cvtColor(A,cv2.COLOR_RGB2GRAY)
nH = int(A.shape[0] // 1.5)
nW = int(A.shape[1] // 1.5)
A = cv2.resize(A, (nW,nH))
for i in range(N_Prueba):
    _, f = capture.read()
    ms = capture.get(0) # Get los milisegundos
    f = cv2.cvtColor(f,cv2.COLOR_RGB2GRAY)
    f = cv2.resize(f, (nW,nH))
    B.append(f)
    mB.append(ms)
capture.release()

In [13]:
print(type(mB[0]))

<class 'float'>


In [9]:
# Retorna un arreglos de ROIs de un frame n, comparando con el frame 0
def roisDeFrame(f0, fn, dimKernelBlur, umbralDif): # BTW: dimKernelBlur debe ser impar
    if len(f0.shape) > 2:
        f0 = cv2.cvtColor(f0,cv2.COLOR_RGB2GRAY)
    if len(fn.shape) > 2:
        fn = cv2.cvtColor(fn,cv2.COLOR_RGB2GRAY)
    D = f0 - fn
    D = cv2.medianBlur(D,dimKernelBlur)
    _, D = cv2.threshold(D,umbralDif,255,cv2.THRESH_BINARY)
    contornos, _ = cv2.findContours(D,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    ROIs = []
    for contorno in contornos:
        x,y,w,h = cv2.boundingRect(contorno)
        ROIs.append((x,y,w,h))
    return ROIs

**Realización:**

In [10]:
# Aquí se realiza el cálculo:
# " *Vxp " significa "Experimentar con los valores"
arrROIs = []
for b in B:
    rois = roisDeFrame(A, b, 27, 180) # * Vxp, la dimensión debe ser impar
    arrROIs.append(rois)

In [11]:
j = 0
capture.open(best.url)
while(True):
    _, frame = capture.read()
    frame = cv2.resize(frame, (nW,nH)) #.... TODO: pasar ROI a la resolución original :c
    if j < len(arrROIs):
        graficarRectDeROIs(frame, arrROIs[j])
    cv2.imshow("Diferencia",frame)
    j += 1
    if(cv2.waitKey(20) & 0xFF == ord('q')):
        break
capture.release()
cv2.destroyAllWindows()

#### Material adicional:

* OpenCV Python Tutorials. *Getting started with videos*. Recuperado de: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
* OpenCV. *Reading and writing images and video* Recuperado de: https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html#videocapture-get 

### Resultados y Discusión (6 puntos)

<font color='green'> Discuta los **varios resultados obtenidos** por la selección de **diversos parámetros seleccionados** </font>

### Conclusiones (2 puntos)

<font color='green'> Redacte, al menos, **5 conclusiones relevantes** referentes a como cubrió su objetivo y que le ayudó a optimizar sus resutlados. </font>